In [27]:
import unittest
import util

# Functions can call other functions

So far, we've only ever encountered functions as a way to take a set of instructions, and seperate them out into an easily repeatable unit of code. Once we've created a function, we've only ever used it by calling it, like below: 

In [1]:
def add_one(x):
    return x + 1

def double(x):
    return x * 2

print(add_one(3))
print(double(3))

4
6


As it turns out though, a function, once created, can be used like any other variable. This means that functions (without the parantheses () ) can be passed to other functions as arguments! Below, we've created a simple function that takes another function as an argument, and prints the answer out

In [2]:
def call_function_on_3(function):
    answer = function(3)
    return answer

print(call_function_on_3(add_one))
print(call_function_on_3(double))

4
6


Note that when we pass a function to another function, we don't include any parantheses. This is because we don't want the function to execute in the line of code where we pass it. Instead, we want call_function_on_3 to call our other function inside of the body of call_function_on_3.

Nifty! But a function that calls another function on the number three is not particularly useful. Let's try something with a little more power. In Python, we're often taking a list and then doing something with every element of the list. We used to write this sort of code by creating an empty list, and then using a *for* loop to grow a new list where each element is transformed by the function

In [3]:
old_list = list(range(5))
output = []
for item in old_list:
    output.append(double(item))
output

[0, 2, 4, 6, 8]

We learned a slightly more fancy way to do this same thing with list comprehensions

In [4]:
output = [double(item) for item in old_list]

Let's write a function that takes a list and another function as arguments, and then outputs a new list where we've applied the function to each element of the old list

In [5]:
def apply_to_all(function_to_apply, old_list):
    output = []
    for old_item in old_list:
        new_item = function_to_apply(old_item)
        output.append(new_item)
    return output

In [6]:
apply_to_all(double, old_list)

[0, 2, 4, 6, 8]

## The map() function

Our apply_to_all function is pretty useful! In fact, it's so useful that python provides a built in version called map(). Unlike our function, however, map() is *lazy*, meaning that it produces a generator. If we don't need the perfomance boost of this generator, however, we can easily turn it into a list.

In [7]:
list(map(double, old_list))

[0, 2, 4, 6, 8]

## The filter() function

filter() is another useful higher order function that Python provides. Basically filter takes a list, and a function that outputs true or false values. Filter() selects only those items that return True from the function. Like map(), it's a function that returns a lazy generator.

In [8]:
def is_even(x):
    return not x % 2

list(filter(is_even, old_list))

[0, 2, 4]

## Sorting with a key

We met the sorted() function last chapter as a way to sort lists. By deafult, sorted() uses the normal comparison of whatever data type is contained in the list. Sometimes, however, we want to customize that way that elements of a list are compared when they're sorted.

Normally, strings are compared using lexical order of the first letter, then the next, then the next. Let's say for some reason, though, we wanted to sort strings by the **last** character, then the next to last, and so on. One way of doing this would be to use map to reverse every string in the list, sort it, and then call map again.

In [9]:
def sort_strings_reverse(strings):
    reversed_strings = [s[::-1] for s in strings]
    reversed_strings = sorted(reversed_strings)
    return [s[::-1] for s in reversed_strings]

In [10]:
sort_strings_reverse(['This', 'is', 'a', 'few', 'strings', 'to', 'sort'])

['a', 'to', 'strings', 'is', 'This', 'sort', 'few']

This is a little tedious, though, and not every sort of transformation we might do on the strings would be so easy to reverse. Instead, sorted() optionally accepts a *key* argument with a function telling it how to sort the items in a list! Let's see how this would work for the example above 

In [11]:
def reverse_string(s):
    return s[::-1]

In [12]:
sorted(['This', 'is', 'a', 'few', 'strings', 'to', 'sort'], key=reverse_string)

['a', 'to', 'strings', 'is', 'This', 'sort', 'few']

Another useful example might be if we're sorting a list of dictionaries, where they keys are different kinds of data. Let's say that we have a list of dictionaries that give information about a city and its temperature:

In [24]:
city_temps = [
    {'city': 'Seattle',
     'temp': 63
    },
    {'city': 'San Francisco',
     'temp': 72
    },
    {'city': 'Los Angeles',
     'temp': 90
    },
    {'city': 'Phoenix',
     'temp': 30000000002
    }
]

There's really two meaningful ways of sorting this data. We could either sort by the city name alphabetically, or by the temperature. Which one does sorted() do by default?

In [14]:
sorted(city_temps)

TypeError: unorderable types: dict() < dict()

It doesn't do either, because in general there's no meaningful way to order two dictionaries! It might look like we're stuck, but sorted() provides an optional second *key* argument that we can use to specify how two objects should be compared. 

The key is a function that maps whatever the input is onto a data type that Python understands comparisons for, like strings or numbers. Let's start by defining a function to pull the city name out of a data point, using dictionary indexing.

In [15]:
def get_city_name(data_point):
    return data_point['city']

get_city_name(city_temps[0])

'Seattle'

Using this function, we can now get a version of the list sorted according to the names of the cities.

In [16]:
sorted(city_temps, key=get_city_name)

[{'city': 'Los Angeles', 'temp': 90},
 {'city': 'Phoenix', 'temp': 30000000000},
 {'city': 'San Francisco', 'temp': 72},
 {'city': 'Seattle', 'temp': 63}]

We can do something similar for the temperatures, extracting them as a number: 

In [17]:
def get_temperature(data_point):
    return data_point['temp']

sorted(city_temps, key=get_temperature)

[{'city': 'Seattle', 'temp': 63},
 {'city': 'San Francisco', 'temp': 72},
 {'city': 'Los Angeles', 'temp': 90},
 {'city': 'Phoenix', 'temp': 30000000000}]

## The lambda keyword

Often times, for functions like map(), filter() and sort(), we find ourselves writing very short one-use functions. While it's generally considered good style to give these functions real names, Python provides a keyword called *lambda* which allows us to quickly define one off functions. Since the use of a greek letter name is a little intimidating, let's just see how we might use lambda to more concisely sort this list by temperature 

In [18]:
sorted(city_temps, key= lambda x: x['temp'])

[{'city': 'Seattle', 'temp': 63},
 {'city': 'San Francisco', 'temp': 72},
 {'city': 'Los Angeles', 'temp': 90},
 {'city': 'Phoenix', 'temp': 30000000000}]

Essentially, lambda is a short way to define functions with a single expression. You define one (at least usually one) variable on the left side of the colon :. On the right side of the colon :, you put a single expression that acts like a return statement. So the expression:

```python
lambda x : x + 1
```

is equivalent to using the following function by name.

```python
def add_one(x):
    return x + 1
```

Lambdas are useful for one-off functions, but it's worth noting that they are extremely limited in what they can do (for example, lambdas cannot contain *if* statements or *for* loops, though they can contain list comprehensions.

If you don't want to use lambda, you can always define a function to do the exact same thing. Many programmers would even consider this to be better style. However, lambdas are quite common in some Python code, and it is good to be familiar with the syntax even if you don't personally use it.

## Aside: Good coding style for accessing attributes of dictionaries

In actual Python code, it's considered somewhat bad practice to write lamdbas for certain common operations, like accesing keys from a dictionary. Python programmers typically use the operator.itemgetter() function to replace lambdas like these, in order to make their code more readable and explicit. You can see it in action below:

In [22]:
import operator

sorted(city_temps, key=operator.itemgetter('city'))

[{'city': 'Los Angeles', 'temp': 90},
 {'city': 'Phoenix', 'temp': 30000000000},
 {'city': 'San Francisco', 'temp': 72},
 {'city': 'Seattle', 'temp': 63}]

In [23]:
sorted(city_temps, key=operator.itemgetter('temp'))

[{'city': 'Seattle', 'temp': 63},
 {'city': 'San Francisco', 'temp': 72},
 {'city': 'Los Angeles', 'temp': 90},
 {'city': 'Phoenix', 'temp': 30000000000}]

# Excercises 

## 1: Implement non-lazy filter() for lists

Above, we implemented the map() function as apply_to_all(). Using a *for* loop, implement the filter() function below as select_items(), returning a list of only the items in collection that meet the criterion of the filter_function.

In [38]:
def select_items(filter_function, collection):
    return [i for i in collection if filter_function(i)] # Could also use a traditional for loop

In [39]:
class FilterTest(unittest.TestCase):
    def test_numerical_filter(self):
        data = [5, 6, 8, 20, 33, 6, 7]
        self.assertEqual(select_items(lambda x: not (x % 2), data), [6, 8, 20, 6])
    def test_long_strings(self):
        data = ['these', 'are', 'a', 'few', 'strings', 'of', 'different', 'lengths']
        self.assertEqual(select_items(lambda x: len(x) > 3, data), ['these', 'strings', 'different', 'lengths'])
        
util.run_tests(FilterTest)

test_long_strings (__main__.FilterTest) ... ok
test_numerical_filter (__main__.FilterTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


## 2: 